In [6]:
import numpy as np 
import pandas as pd 
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_selection import SelectKBest, chi2
from xgboost import XGBClassifier
from sklearn.model_selection import cross_val_score


import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))


/kaggle/input/open-minilm-l6-v2/rust_model.ot
/kaggle/input/open-minilm-l6-v2/config.json
/kaggle/input/open-minilm-l6-v2/tokenizer.json
/kaggle/input/open-minilm-l6-v2/tf_model.h5
/kaggle/input/open-minilm-l6-v2/data_config.json
/kaggle/input/open-minilm-l6-v2/train_script.py
/kaggle/input/open-minilm-l6-v2/tokenizer_config.json
/kaggle/input/open-minilm-l6-v2/sentence_bert_config.json
/kaggle/input/open-minilm-l6-v2/gitattributes
/kaggle/input/open-minilm-l6-v2/pytorch_model.bin
/kaggle/input/open-minilm-l6-v2/config_sentence_transformers.json
/kaggle/input/open-minilm-l6-v2/model.safetensors
/kaggle/input/open-minilm-l6-v2/modules.json
/kaggle/input/open-minilm-l6-v2/special_tokens_map.json
/kaggle/input/open-minilm-l6-v2/vocab.txt
/kaggle/input/llm-classification-finetuning/sample_submission.csv
/kaggle/input/llm-classification-finetuning/train.csv
/kaggle/input/llm-classification-finetuning/test.csv


In [7]:


#!pip install -q sentence-transformers

from sentence_transformers import SentenceTransformer

#model = SentenceTransformer("all-mpnet-base-v2")


In [8]:

#model.save("/kaggle/working/all-mpnet-base-v2")


In [9]:
from transformers import AutoModel, AutoTokenizer

#tokenizer = AutoTokenizer.from_pretrained("/kaggle/input/qwen-llm")
#model = AutoModel.from_pretrained("/kaggle/input/qwen-llm")



In [10]:
submission_test = pd.read_csv("/kaggle/input/llm-classification-finetuning/sample_submission.csv")
submission_test

,id,winner_model_a,winner_model_b,winner_tie
0,136060,0.333333,0.333333,0.333333
1,211333,0.333333,0.333333,0.333333
2,1233961,0.333333,0.333333,0.333333


In [11]:
df_train = pd.read_csv("/kaggle/input/llm-classification-finetuning/train.csv")
df_train

,id,model_a,model_b,prompt,response_a,response_b,winner_model_a,winner_model_b,winner_tie
0,30192,gpt-4-1106-preview,gpt-4-0613,"[""Is it morally right to try to have a certain...","[""The question of whether it is morally right ...","[""As an AI, I don't have personal beliefs or o...",1,0,0
1,53567,koala-13b,gpt-4-0613,"[""What is the difference between marriage lice...","[""A marriage license is a legal document that ...","[""A marriage license and a marriage certificat...",0,1,0
2,65089,gpt-3.5-turbo-0613,mistral-medium,"[""explain function calling. how would you call...","[""Function calling is the process of invoking ...","[""Function calling is the process of invoking ...",0,0,1
3,96401,llama-2-13b-chat,mistral-7b-instruct,"[""How can I create a test set for a very rare ...","[""Creating a test set for a very rare category...","[""When building a classifier for a very rare c...",1,0,0
4,198779,koala-13b,gpt-3.5-turbo-0314,"[""What is the best way to travel from Tel-Aviv...","[""The best way to travel from Tel Aviv to Jeru...","[""The best way to travel from Tel-Aviv to Jeru...",0,1,0
...,...,...,...,...,...,...,...,...,...
57472,4294656694,gpt-4-0613,claude-1,"[""A simple mnemonic for \u03c0:\n\""How I wish ...","[""Sure, let's break it down:\n\n1. \""How\"" has...","[""Here is how that mnemonic represents the dig...",1,0,0
57473,4294692063,claude-2.0,llama-2-13b-chat,"[""In python, implement a naive Bayes with gaus...","[""Here is an implementation of a naive Bayes c...","[""Sure! Here's an implementation of a naive Ba...",1,0,0
57474,4294710549,claude-1,alpaca-13b,"[""is it unethical to work on building weapons?...","[""Working on weapons technology raises some et...","[""It depends on the context. Weapons can be us...",1,0,0
57475,4294899228,palm-2,tulu-2-dpo-70b,"[""If a bait contains 0,0025% bromadiolon then ...","[""Bromadiolone is a rodenticide which is most ...","[""As an AI language model, I do not promote or...",0,1,0


In [12]:
df_test = pd.read_csv("/kaggle/input/llm-classification-finetuning/test.csv")
df_test

,id,prompt,response_a,response_b
0,136060,"[""I have three oranges today, I ate an orange ...","[""You have two oranges today.""]","[""You still have three oranges. Eating an oran..."
1,211333,"[""You are a mediator in a heated political deb...","[""Thank you for sharing the details of the sit...","[""Mr Reddy and Ms Blue both have valid points ..."
2,1233961,"[""How to initialize the classification head wh...","[""When you want to initialize the classificati...","[""To initialize the classification head when p..."


In [13]:
df_train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 57477 entries, 0 to 57476
Data columns (total 9 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   id              57477 non-null  int64 
 1   model_a         57477 non-null  object
 2   model_b         57477 non-null  object
 3   prompt          57477 non-null  object
 4   response_a      57477 non-null  object
 5   response_b      57477 non-null  object
 6   winner_model_a  57477 non-null  int64 
 7   winner_model_b  57477 non-null  int64 
 8   winner_tie      57477 non-null  int64 
dtypes: int64(4), object(5)
memory usage: 3.9+ MB


In [14]:
df_train.isnull().sum()

id                0
model_a           0
model_b           0
prompt            0
response_a        0
response_b        0
winner_model_a    0
winner_model_b    0
winner_tie        0
dtype: int64

In [15]:
X = df_train.drop(['model_a', 'model_b', 'winner_model_a', 'winner_model_b', 'winner_tie'], axis = 1)
X

,id,prompt,response_a,response_b
0,30192,"[""Is it morally right to try to have a certain...","[""The question of whether it is morally right ...","[""As an AI, I don't have personal beliefs or o..."
1,53567,"[""What is the difference between marriage lice...","[""A marriage license is a legal document that ...","[""A marriage license and a marriage certificat..."
2,65089,"[""explain function calling. how would you call...","[""Function calling is the process of invoking ...","[""Function calling is the process of invoking ..."
3,96401,"[""How can I create a test set for a very rare ...","[""Creating a test set for a very rare category...","[""When building a classifier for a very rare c..."
4,198779,"[""What is the best way to travel from Tel-Aviv...","[""The best way to travel from Tel Aviv to Jeru...","[""The best way to travel from Tel-Aviv to Jeru..."
...,...,...,...,...
57472,4294656694,"[""A simple mnemonic for \u03c0:\n\""How I wish ...","[""Sure, let's break it down:\n\n1. \""How\"" has...","[""Here is how that mnemonic represents the dig..."
57473,4294692063,"[""In python, implement a naive Bayes with gaus...","[""Here is an implementation of a naive Bayes c...","[""Sure! Here's an implementation of a naive Ba..."
57474,4294710549,"[""is it unethical to work on building weapons?...","[""Working on weapons technology raises some et...","[""It depends on the context. Weapons can be us..."
57475,4294899228,"[""If a bait contains 0,0025% bromadiolon then ...","[""Bromadiolone is a rodenticide which is most ...","[""As an AI language model, I do not promote or..."


In [16]:
y = df_train[['winner_model_a', 'winner_model_b', 'winner_tie']].values

y = np.argmax(y, axis=1)
y

array([0, 1, 2, ..., 0, 1, 0])

In [17]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size =0.2, random_state = 42)

In [18]:
X_train

,id,prompt,response_a,response_b
30205,2245506736,"[""can you give me some escape room themes with...","[""Certainly! Here are several escape room them...","[""Here are some fun and immersive escape room ..."
8763,656268525,"[""Why did digital electronics switch from 5 vo...","[""There are a few key reasons why digital elec...","[""The shift from 5 volt logic to 3 volt logic ..."
30501,2266396135,"[""as an acclaimed and visionary author punishe...","[""As an acclaimed and visionary author, Elieze...","[""Eliezer Yudkowsky is often referred to as th..."
33619,2497468695,"[""\n\nWrite a micro editor plugin that shows t...","[""Here is Python code for a basic micro editor...","[""Micro is a modern and intuitive terminal-bas..."
22573,1679992057,"[""1. 19 year old Emese who is thin (165cm 48kg...","[""Certainly, let's address each of your points...","[""1. From the description you provided, it see..."
...,...,...,...,...
54343,4061186081,"[""mega-sena brazilian lottery statistics""]","[""I do not have access to real-time mega-sena ...","[""Mega-Sena is a Brazilian lottery game that o..."
38158,2840606458,"[""Draw a dog using ASCII art""]","[""\/_\/\\ \n( o.o )\n> ^""]","[""Here is a dog drawn with ASCII art:\n\n ..."
860,62750940,"[""How to wear a bikini?""]","[""Wearing a bikini can be a fun and comfortabl...","[""Here are the basic steps to wear a bikini:\n..."
15795,1180878567,"[""Can you tell me what was going on with all t...","[""The direwolves, which are also known as dire...","[""Sure! On the night of the Red Wedding, all o..."


In [19]:
X_test

,id,prompt,response_a,response_b
37379,2785062085,"[""what does hello world mean""]","[""\""Hello, World!\"" is a phrase used in comput...","[""\""Hello, World!\"" is a common phrase used to..."
644,48259531,"[""I ran a marathon in 3:12:00 weighting 84kg. ...","[""It's difficult to provide an exact answer to...","[""To accurately estimate how much faster you w..."
48496,3622731894,"[""Below is an instruction that describes a tas...","[""\u8fd9\u6b3e\u7eaf\u8272\u4e94\u5206\u88e4\u...","[""\u6839\u636e\u63d0\u4f9b\u7684\u4ea7\u54c1\u..."
12603,933663766,"[""How do I run static analysis with gcc in QT ...","[""To run static analysis with gcc in QT Creato...","[""To run static analysis with gcc in QT Creato..."
16697,1246768370,"[""What did the music producer \u00d8fdream die...","[""I am not able to provide information on the ...","[""I'm really sorry, but I don't have the abili..."
...,...,...,...,...
3151,240261313,"[""How to save a web page into a jpg file?""]","[""Here are a few ways to save a web page as a ...","[""Here are a few ways to save a web page as a ..."
30983,2300512815,"[""provide a short story about a pirate that le...","[""Here is a short story about a pirate who lea...","[""Once upon a time, in the golden age of pirac..."
31183,2314277410,"[""Name 3 features of Alain Badiou's ontology t...","[""Alain Badiou and Plato have distinct philoso...","[""Here are 3 key differences between Alain Bad..."
50634,3781372084,"[""Please give me a list of concise bullet poin...","[""- Musician who plays guitar and enjoys jammi...","[""* Music: plays guitar, drums, and bass\n* Sp..."


In [20]:
y_train

array([2, 1, 1, ..., 1, 1, 0])

In [21]:
catagorical_feature = [col for col in X.columns if X[col].dtype == 'object']
catagorical_feature

['prompt', 'response_a', 'response_b']

In [20]:
from sentence_transformers import SentenceTransformer
from sklearn.base import BaseEstimator, TransformerMixin

from sklearn.compose import ColumnTransformer

from sklearn.pipeline import Pipeline
from sklearn.feature_selection import SelectKBest, chi2


from sentence_transformers import SentenceTransformer
import torch

class HFEmbedder(BaseEstimator, TransformerMixin):
    def __init__(self, model_name="all-mpnet-base-v2"):
        self.model_name = model_name  
        self.model = SentenceTransformer(model_name)
        if torch.cuda.is_available():
            self.model = self.model.to("cuda")

    def fit(self, X, y=None):
        return self

    def transform(self, X):
        return self.model.encode(X.tolist(), show_progress_bar=False, device="cuda" if torch.cuda.is_available() else "cpu")


# Example usage in ColumnTransformer
preprocessing = ColumnTransformer([
    ("prompt_embed", HFEmbedder(), "prompt"),
    ("resp_a_embed", HFEmbedder(), "response_a"),
    ("resp_b_embed", HFEmbedder(), "response_b"),
    ("num", "passthrough", ["id"])
])


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md: 0.00B [00:00, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/571 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/363 [00:00<?, ?B/s]

vocab.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/239 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

from sentence_transformers import SentenceTransformer
from sklearn.base import BaseEstimator, TransformerMixin

from sklearn.compose import ColumnTransformer

from sklearn.pipeline import Pipeline
from sklearn.feature_selection import SelectKBest, chi2


from sentence_transformers import SentenceTransformer
import torch
import torch

class HFEmbedder(BaseEstimator, TransformerMixin):
    def __init__(self, model_path="/kaggle/input/open-minilm-l6-v2",use_auth_token=False , local_files_only=True,batch_size=8, use_cuda=True):
        self.model_path = model_path
        self.batch_size = batch_size
        self.use_cuda = use_cuda and torch.cuda.is_available()
        self.model = SentenceTransformer(self.model_path)
        if self.use_cuda:
            self.model = self.model.to("cuda")

    def fit(self, X, y=None):
        return self

    def transform(self, X):
        texts = X.tolist()
        all_embeddings = []

        for i in range(0, len(texts), self.batch_size):
            batch = texts[i:i + self.batch_size]
            embeddings = self.model.encode(
                batch,
                show_progress_bar=False,
                device="cuda" if self.use_cuda else "cpu"
            )
            all_embeddings.append(embeddings)

        return np.vstack(all_embeddings)



In [37]:
from transformers import AutoTokenizer, AutoModel
from sklearn.base import BaseEstimator, TransformerMixin
import torch
import numpy as np

class HFEmbedder(BaseEstimator, TransformerMixin):
    def __init__(self, model_path="/kaggle/input/open-minilm-l6-v2", batch_size=8, use_cuda=True):
        self.model_path = model_path
        self.batch_size = batch_size
        self.use_cuda = use_cuda and torch.cuda.is_available()

        self.tokenizer = AutoTokenizer.from_pretrained(self.model_path)
        self.model = AutoModel.from_pretrained(self.model_path)
        if self.use_cuda:
            self.model = self.model.to("cuda")

    def fit(self, X, y=None):
        return self

    def transform(self, X):
        texts = X.tolist()
        all_embeddings = []

        for i in range(0, len(texts), self.batch_size):
            batch = texts[i:i + self.batch_size]
            inputs = self.tokenizer(batch, padding=True, truncation=True, return_tensors="pt")
            if self.use_cuda:
                inputs = {k: v.to("cuda") for k, v in inputs.items()}
            with torch.no_grad():
                outputs = self.model(**inputs)
            # Mean pooling over token embeddings
            embeddings = outputs.last_hidden_state.mean(dim=1).cpu().numpy()
            all_embeddings.append(embeddings)

        return np.vstack(all_embeddings)


In [34]:
print(os.listdir("/kaggle/input/open-minilm-l6-v2"))


['rust_model.ot', 'config.json', 'tokenizer.json', 'tf_model.h5', 'data_config.json', 'train_script.py', 'tokenizer_config.json', 'sentence_bert_config.json', 'gitattributes', 'pytorch_model.bin', 'config_sentence_transformers.json', 'model.safetensors', 'modules.json', 'special_tokens_map.json', 'vocab.txt']


In [38]:


preprocessing = ColumnTransformer([
    ("prompt_embed", HFEmbedder("/kaggle/input/open-minilm-l6-v2"), "prompt"),
    ("resp_a_embed", HFEmbedder("/kaggle/input/open-minilm-l6-v2"), "response_a"),
    ("resp_b_embed", HFEmbedder("/kaggle/input/open-minilm-l6-v2"), "response_b"),
    ("num", "passthrough", ["id"])
])


In [39]:
preprocessing

ColumnTransformer(transformers=[('prompt_embed', HFEmbedder(), 'prompt'),
                                ('resp_a_embed', HFEmbedder(), 'response_a'),
                                ('resp_b_embed', HFEmbedder(), 'response_b'),
                                ('num', 'passthrough', ['id'])])

In [40]:
#feature_selection = SelectKBest(score_func = chi2, k=6)
from sklearn.feature_selection import SelectKBest, f_classif

feature_selection = SelectKBest(score_func=f_classif, k=6)


In [41]:
model = XGBClassifier(
    objective="multi:softprob",  
    num_class=3,                  
    eval_metric="mlogloss",       
    n_estimators=300,
    learning_rate=0.1,
    max_depth=6,
    random_state=42
)

In [42]:
my_pipeline = Pipeline([
    ("preprocessing", preprocessing),
    #("feature_selection", feature_selection),
    ("model", model)
])
my_pipeline

Pipeline(steps=[('preprocessing',
                 ColumnTransformer(transformers=[('prompt_embed', HFEmbedder(),
                                                  'prompt'),
                                                 ('resp_a_embed', HFEmbedder(),
                                                  'response_a'),
                                                 ('resp_b_embed', HFEmbedder(),
                                                  'response_b'),
                                                 ('num', 'passthrough',
                                                  ['id'])])),
                ('model',
                 XGBClassifier(base_score=None, booster=None, callbacks=None,
                               colsample_bylevel=None, colsample_bynode=None,
                               colsample_bytree...
                               feature_types=None, gamma=None, grow_policy=None,
                               importance_type=None,
                               interaction_constraints=None, learning_rate=0.1,
                               max_bin=None, max_cat_threshold=None,
                               max_cat_to_onehot=None, max_delta_step=None,
                               max_depth=6, max_leaves=None,
                               min_child_weight=None, missing=nan,
                               monotone_constraints=None, multi_strategy=None,
                               n_estimators=300, n_jobs=None, num_class=3,
                               num_parallel_tree=None, ...))])

In [43]:
scores = cross_val_score(my_pipeline, X_train, y_train, cv=3, scoring="accuracy")
print("Accuracy:", scores.mean())

Accuracy: 0.4487940671146778


In [44]:
my_pipeline.fit(X_train, y_train)

Pipeline(steps=[('preprocessing',
                 ColumnTransformer(transformers=[('prompt_embed', HFEmbedder(),
                                                  'prompt'),
                                                 ('resp_a_embed', HFEmbedder(),
                                                  'response_a'),
                                                 ('resp_b_embed', HFEmbedder(),
                                                  'response_b'),
                                                 ('num', 'passthrough',
                                                  ['id'])])),
                ('model',
                 XGBClassifier(base_score=None, booster=None, callbacks=None,
                               colsample_bylevel=None, colsample_bynode=None,
                               colsample_bytree...
                               feature_types=None, gamma=None, grow_policy=None,
                               importance_type=None,
                               interaction_constraints=None, learning_rate=0.1,
                               max_bin=None, max_cat_threshold=None,
                               max_cat_to_onehot=None, max_delta_step=None,
                               max_depth=6, max_leaves=None,
                               min_child_weight=None, missing=nan,
                               monotone_constraints=None, multi_strategy=None,
                               n_estimators=300, n_jobs=None, num_class=3,
                               num_parallel_tree=None, ...))])

In [45]:
probs = my_pipeline.predict_proba(df_test)

submission = pd.DataFrame({
    "id": df_test["id"],
    "winner_model_a": probs[:,0],
    "winner_model_b": probs[:,1],
    "winner_tie": probs[:,2],
})
print(submission)
submission.to_csv("submission.csv", index=False)

        id  winner_model_a  winner_model_b  winner_tie
0   136060        0.373232        0.359567    0.267201
1   211333        0.341727        0.248099    0.410174
2  1233961        0.242470        0.482838    0.274693
